In [1]:
!pip install -U sentence-transformers
!pip install torch
!pip install json
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [2]:
from sentence_transformers import SentenceTransformer, util
import json
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Define a function to load JSON files
def load_json_file(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

# paths to the files
topic_filepath = 'topics_1.json'
answer_filepath = 'Answers.json'

In [5]:
import re
import bs4
def clean_topics_text(item):

    full_string = item['Title'] + ' ' + item['Body'] + ' ' + item['Tags']

    full_string = bs4.BeautifulSoup(full_string, 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string

def clean_answers_text(item):
    full_string = bs4.BeautifulSoup(item['Text'], 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string


# Clean em' up
topics = load_json_file(topic_filepath)
answers = load_json_file(answer_filepath)

cleaned_topics = [clean_topics_text(topic) for topic in topics]
cleaned_answers = [clean_answers_text(answer) for answer in answers]


Why do flight attendants say in safety instructions that you should put on your own mask before helping others  I have taken several flights from Emirates  FlyDubai and Saudi Airlines  Why is it announced to put your own mask on before helping others with their mask  I wouldn t do it if I am travelling with my family  I would like to help them first  So what s the reason    air travel    safety    airlines  
I have seen two dissenting opinions on this subject  and in my opinion  the first option is right  By definition  a Sudoku has only one solution  Anything else is just a grid of numbers  Sometimes  there are errors in a publication  and a starting grid has multiple solutions  but  then the starting grid was not a Sudoku From Wikipedia  The number of classic     Sudoku solution grids is                                or approximately            ignoring rotations and other factors  there are many fewer solutions  just              Assuming you define a sudoku to have just one possib

In [6]:

bi_encoder = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
topic_embeddings = bi_encoder.encode(cleaned_topics, convert_to_tensor=True)
answer_embeddings = bi_encoder.encode(cleaned_answers, convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
torch.save(topic_embeddings, 'topic_embeddings.pt')
torch.save(answer_embeddings, 'answer_embeddings.pt')

# Optionally, save the cosine similarity matrix
cos_sim = util.cos_sim(topic_embeddings, answer_embeddings)
torch.save(cos_sim, 'cos_sim.pt')

In [11]:
# Step 1: Load the saved embeddings
topic_embeddings = torch.load('topic_embeddings.pt')
answer_embeddings = torch.load('answer_embeddings.pt')

# Step 2: Compute cosine similarity between topic embeddings and answer embeddings
cos_sim = util.cos_sim(topic_embeddings, answer_embeddings)

# Step 3: Get the top 100 results for each topic
top_k = 100
top_results = []

# Assuming you have the original topics and answers dictionaries
topics = load_json_file('topics_1.json')
answers = load_json_file('Answers.json')

for i in range(len(topics)):
    # Get the top_k answers for the current topic
    top_k_values, top_k_indices = torch.topk(cos_sim[i], k=top_k)
    topic_results = [(answers[idx], val.item()) for idx, val in zip(top_k_indices, top_k_values)]
    top_results.append(topic_results)

# Step 4: Save the top results to a file
with open('top_100_results.json', 'w') as f:
    json.dump(top_results, f)

<ipython-input-11-2a1ec05fecce>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  topic_embeddings = torch.load('topic_embeddings.pt')
<ipython-input-11-2a1ec05fecce>:3: Futu